# - Analizing info. -

In [73]:
import pandas as pd
from pandas import json_normalize

import requests 
import json
import os
from dotenv import load_dotenv #conda install -c conda-forge python-dotenv

import time
import importlib
import src.extraction as ex
importlib.reload(ex)

<module 'src.extraction' from 'd:\\ironhack\\proyectos\\PipeLine_proy2\\src\\extraction.py'>

## 1. USDA food database.
### 1.2. Exploration.

Did a research of all the USDA csv files and select only the more relevant for this proyect, they have a lot of information about comples dishes, lab tests, market prices, etc.
    1. Searching for the right tables with value information for the proyect.
    2. Cleaning each table, droping the columns and probably some rows with no need it information.
    ----try to merge it in python by changing the name of tha columns to the same in each. 
        exported as csv
    3. Merge the tables -- prbably on SQL 
        exported again in csv and put it back on python

####   1.2.1. "Food table" 
Data frame of food, it has its id number and name of food.

In [2]:
food = "D:\ironhack\proyectos\PipeLine_proy2\\USDA_main_csv\\food.csv"

In [69]:
extraction.import_csv(food)

,fdc_id,data_type,description,food_category_id,publication_date
0,1105904,branded_food,WESSON Vegetable Oil 1 GAL,NaN,2020-11-13
1,1105905,branded_food,SWANSON BROTH BEEF,NaN,2020-11-13
2,1105906,branded_food,CAMPBELL'S SLOW KETTLE SOUP CLAM CHOWDER,NaN,2020-11-13
3,1105907,branded_food,CAMPBELL'S SLOW KETTLE SOUP CHEESE BROCCOLI,NaN,2020-11-13
4,1105898,experimental_food,Discrepancy between the Atwater factor predict...,NaN,2020-10-30
...,...,...,...,...,...
1185085,1757386,sub_sample_food,NaN,NaN,2021-04-23
1185086,1757387,sub_sample_food,NaN,NaN,2021-04-23
1185087,1757388,sub_sample_food,NaN,NaN,2021-04-23
1185088,1757389,sub_sample_food,NaN,NaN,2021-04-23


In [13]:
food = pd.read_csv("D:\ironhack\proyectos\PipeLine_proy2\\USDA_main_csv\\food.csv")
food.sample(5)

,fdc_id,data_type,description,food_category_id,publication_date
819157,1391456,branded_food,SALTED CARAMEL RICH SALTED CARAMEL GELATO WITH...,NaN,2021-02-26
870697,1442996,branded_food,"DRY ROASTED SALTED ORGANIC PISTACHIOS, DRY ROA...",NaN,2021-02-26
845586,1417885,branded_food,"100% WHITE GRAPE JUICE FROM CONCENTRATE, WHITE...",NaN,2021-02-26
526173,1031653,branded_food,HONDURAN MEDIUM ROAST NATURAL CHOCOLATE FLAVOR...,NaN,2020-06-26
902091,1474390,branded_food,"B&G, GHERKINS SWEET PICKLES",NaN,2021-03-19


In [14]:
food.shape

(1185090, 5)

##### 1.2.1.1. Deleting  
Will remove the "food_category_id", "data_type" and the "publication_date" column and try to remove the index column, it appears that it does not do much, it doest have a column name for identification.

In [15]:
food.drop(columns = ["food_category_id", "publication_date", "data_type"], inplace=True, axis=1)
food.sample(5)

,fdc_id,description
240064,527631,MILK SHAKERS
1020365,1592664,BIG PEPPER JACK CHEESE SNACK
1016091,1588390,HEIRLOOM HOT SAUCE WITH CO-OP SAUCE
816219,1388518,BROWNIE BITES
97733,385300,BUTTER


Remove the null values, theres only 9 rows will delete.

In [202]:
def delete_1 (x):
    # return the cleaining of columns from "food" data base
    x.drop(columns = ["food_category_id", "publication_date", "data_type"], inplace=True, axis=1)
    x.dropna(how="any", inplace=True)
    return x

In [16]:
food.isnull().sum()

fdc_id         0
description    9
dtype: int64

In [17]:
food.dropna(how="any", inplace=True)
food.sample(5)

,fdc_id,description
1070421,1642720,"THE MADELAINE CHOCOLATE COMPANY, SOLID MILK CH..."
797598,1369897,"LIGHTLY ROASTED ALMONDS, LIGHTLY ROASTED"
12139,1118037,"KOSHER FLAVORED PICKLE, KOSHER"
385953,742066,"REDUCED SODIUM CHICKEN BROTH, CHICKEN"
1140722,1713021,FALL MAPLE FLAVORED WITH A BLEND OF ROASTED SA...


In [18]:
food.duplicated().sum()

0

In [19]:
food.shape

(1185081, 2)

#### 1.2.2. "Food nutrients table"
In this table can relate the data of nutrients and in wich food they are. they specifie the mount of the nutrient per 100g of food. Specified in unit defined in the nutrient
table.

In [20]:
food_nutrients = pd.read_csv("D:\ironhack\proyectos\PipeLine_proy2\\USDA_main_csv\\food_nutrient.csv")
food_nutrients.sample(3)

C:\Users\ima_n\AppData\Local\Temp\ipykernel_21760\1622644252.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  food_nutrients = pd.read_csv("D:\ironhack\proyectos\PipeLine_proy2\\USDA_main_csv\\food_nutrient.csv")


,id,fdc_id,nutrient_id,amount,data_points,derivation_id,min,max,median,footnote,min_year_acquired
11925237,17715985,1558396,601,9.00,NaN,70.0,NaN,NaN,NaN,NaN,NaN
10550853,16341601,1434520,204,10.53,NaN,70.0,NaN,NaN,NaN,NaN,NaN
8126008,13916756,1150940,291,1.40,NaN,73.0,NaN,NaN,NaN,NaN,NaN


In [21]:
food_nutrients.shape

(15324912, 11)

##### 1.2.2.1. Deleting
Need to identified which columns I wont need, in this table I have to important values that will conect my tables with each other and is the "fdc_id" and "nutrient_id" and its own "id" column, we can see also that theres a lot of null columns that dont apport nothing.

In [22]:
food_nutrients.isnull().sum()

id                          0
fdc_id                      0
nutrient_id                 0
amount                      0
data_points          14577366
derivation_id          642147
min                  15218166
max                  15218164
median               15314455
footnote             15324645
min_year_acquired    15300052
dtype: int64

We can see that in some columns, most of the information is null, we will delete them.

In [23]:
food_nutrients.drop(columns = ["data_points", "derivation_id", "min", "max",  "median", "footnote", "min_year_acquired"], inplace=True, axis=1)
food_nutrients.sample(3)

,id,fdc_id,nutrient_id,amount
708143,2281245,335686,1094,238.00
4138866,6301783,352684,1093,54.00
3902937,6065854,483450,1003,10.57


In [24]:
food_nutrients.isnull().sum()

id             0
fdc_id         0
nutrient_id    0
amount         0
dtype: int64

#### 1.2.3. "Nutrients Table"

In [25]:
nutrient = pd.read_csv("D:\ironhack\proyectos\PipeLine_proy2\\USDA_main_csv\\nutrient.csv")
nutrient.sample(3)

,id,name,unit_name,nutrient_nbr,rank
411,2000,"Sugars, total including NLEA",G,269.0,1510.0
362,1366,Epicatechin,MG,751.0,20500.0
417,2008,MUFA 12:1,G,635.0,11450.0


##### 1.2.3.1 Deleting.
will delete must of the columns, except name and id, but will create a copy so i dont lose the information, could help me in the future.

In [26]:
nutrient.isnull().sum()

id               0
name             0
unit_name        0
nutrient_nbr     4
rank            11
dtype: int64

In [27]:
nutrient.shape

(463, 5)

In [28]:
nutrient_comp = nutrient.copy

In [203]:
def delete_1 (x):
    # return the cleaining of columns from "nutrient" data base
    x.drop(columns = ["unit_name", "nutrient_nbr", "rank"], inplace= True, axis=1)    
    x.dropna(how="any", inplace=True)
    return x

In [29]:
nutrient.drop(columns = ["unit_name", "nutrient_nbr", "rank"], inplace= True, axis=1)
nutrient.sample()

,id,name
328,1332,SFA 13:0


In [30]:
nutrient.duplicated().sum()

0

For now will not use this table, it says at the documentetion that is to categorize the nutrients in a big scale, each of them are related to a nutrient.

In [31]:
nutrient_incoming_name = pd.read_csv("D:\ironhack\proyectos\PipeLine_proy2\\USDA_main_csv\\nutrient_incoming_name.csv")
nutrient_incoming_name.sample(5)

,id,name,nutrient_id
646,1667,Heptanoic acid,2004
176,1186,Vitamin D (IU),1110
665,1706,Lutein/Zeaxanthin,1123
508,1520,C22:5 Docosapentaenoic,1280
83,1087,"MOISTURE, 60 DEGREE",1051


In [32]:
nutrient_incoming_name.shape

(730, 3)

This table doesn´t appear in the information docs, but it loocks really helfull allready matching the nutrient with the food and the portion per 100g. Wont delete any column, but will start to change names of each Data Frame to matched them later.

In [33]:
FoodId_NutrientId = pd.read_csv("D:\ironhack\proyectos\PipeLine_proy2\\USDA_main_csv\\JCS_FoodId-NutrientId-Per100g.csv")
FoodId_NutrientId.sample(5)

,FoodId,NutrientId,Per100g
3550467,557889,1257,0.00
1794525,404717,1258,0.00
4305161,612820,1093,211.00
14103326,1705099,606,20.83
1443482,485688,1089,0.00


In [34]:
FoodId_NutrientId.shape

(15324912, 3)

In [35]:
from IPython.display import display_html 

### 1.2. Merging
Will merge all data frames into one, will chose the right tables for it and clean it more.

Looking for same elements in different tables at the ID rows.

In [36]:
FoodId_NutrientId[FoodId_NutrientId['FoodId'] == '631798'].index

Int64Index([], dtype='int64')

In [37]:
food[food['fdc_id'] == '631798'].index

Int64Index([], dtype='int64')

In [38]:
FoodId_NutrientId[FoodId_NutrientId['NutrientId'] == '1304'].index

Int64Index([], dtype='int64')

In [39]:
nutrient[nutrient['id'] == '1304'].index

Int64Index([], dtype='int64')

#### 1.2.1. Renaming
Replace the column names with a same name in each table, for better merging.

In [40]:
# change decription to food
# change fdc_id to food_id
food.sample()

,fdc_id,description
796807,1369106,ORIGINAL CREAM CHEESE


In [41]:
food.rename(columns= {"fdc_id" : "food_id", "description" : "food"}, inplace=True)
food.sample()

,food_id,food
1012615,1584914,TAHINI SESAME PASTE


In [204]:
def rename_columns (food, nutrients, FoodId_NutrientId):
    food.rename(columns= {"fdc_id" : "food_id", "description" : "food"}, inplace=True)
    FoodId_NutrientId.rename(columns= {"FoodId" : "food_id", "NutrientId" : "nutrient_id", 'Per100g ' : "Per100g"}, inplace=True)
    nutrient.rename(columns= {"name" : "nutrient", "id" : "nutrient_id"}, inplace=True)
    return food, nutrients, FoodId_NutrientId

In [206]:
rename_columns (food, nutrients, FoodId_NutrientId)

NameError: name 'nutrients' is not defined

In [42]:
#change NutrientId to nutrient_id
#change foodId to food_id
FoodId_NutrientId.sample()

,FoodId,NutrientId,Per100g
7624193,1109520,601,0.0


In [43]:
FoodId_NutrientId.rename(columns= {"FoodId" : "food_id", "NutrientId" : "nutrient_id", 'Per100g ' : "Per100g"}, inplace=True)
FoodId_NutrientId.sample()

,food_id,nutrient_id,Per100g
6894449,1070267,1079,1.6


In [44]:
# change id to nutrient_id
# change name to nutrient
nutrient.sample()

,id,name
448,2041,Tocopherols and tocotrienols


In [45]:
nutrient.rename(columns= {"name" : "nutrient", "id" : "nutrient_id"}, inplace=True)
nutrient.sample()

,nutrient_id,nutrient
207,1210,Tryptophan


#### 1.2.2. Merge

In [46]:
list(inner_merge.columns)

NameError: name 'inner_merge' is not defined

In [ ]:
inner_merge = pd.merge(nutrient, FoodId_NutrientId, on="nutrient_id", how="inner")
inner_merge.sample()

,nutrient_id,nutrient,food_id,Per100g
6095097,1257,"Fatty acids, total trans",929406,0.0


In [ ]:
def inner_merge ():

In [ ]:
usda = pd.merge(inner_merge, food, on="food_id", how="inner")
usda.sample()

,nutrient_id,nutrient,food_id,Per100g,food
6886894,1008,Energy,1080403,393.0,"PEPPERMINT, COOKIE, EGG NOG & CRANBERRY ASSORT..."


Re-arrange the column in a better order for mor understanding.

In [ ]:
usda = usda[["food", "nutrient", "Per100g", "food_id", "nutrient_id" ]]
usda.sample(5)

,food,nutrient,Per100g,food_id,nutrient_id
4899668,WILD ALASKA CHUM SALMON,"Fiber, total dietary",0.00,704888,1079
4012806,8/10 OCTOPUS,"Vitamin A, IU",0.00,352235,1104
1207134,SPICED COOKIE SPREAD,"Carbohydrate, by difference",56.76,532102,1005
1011738,"FLAVE!, ASSORTED GUMMY BEARS, 12 FLAVORS","Sugars, total including NLEA",47.50,401042,2000
7452218,"Beer, light, higher alcohol",Vitamin B-12,0.02,1104396,1178


In [ ]:
# Clean list of nutrients
list(usda["nutrient"])
usda["nutrient"] = [i.strip().replace("  ", " ") for i in list(usda["nutrient"])]


In [ ]:
print(list(usda["nutrient"]))

NameError: name 'usda' is not defined

In [ ]:
whey = usda[usda["food"]=="CEREAL"]
whey.sample(50)


ValueError: Cannot take a larger sample than population when 'replace=False'

Will need to do a bigger filter, theres a lot of the same food name but still different for their id number, it probably is because its same product of different brand. Still we weally dont need it for this proyect.
Drop the duplicated.

In [ ]:
usda.drop(columns = ["Per100g", "food_id", "nutrient_id"], inplace= True, axis=1)
usda.sample()

,food,nutrient
7404602,"Relish, corn","Fatty acids, total monounsaturated"


In [ ]:
usda.duplicated().sum()

2822490

In [ ]:
usda.shape

(7555511, 2)

In [ ]:
usda.drop_duplicates(inplace=True)
usda.shape

(4733021, 2)

In [ ]:
usda.sample()

,food,nutrient
3449723,"HARRIS TEETER,SWEETENED CONDENSED MILK","Vitamin C, total ascorbic acid"


In [ ]:
# relacionarlos con porcentaje de nutrients, count() sum()
# revisar el type() de cada columna - strings
usda_group = usda.groupby(by=["food"])
usda_group

In [ ]:
# relacionarlos con porcentaje de nutrients, count() sum()
usda[["food", "nutrient"]].groupby(by=["food"]).agg({"nutrient":"value_counts"})


nutrient
food        nutrient                                
 Egg whites Calcium, Ca                            1
            Copper, Cu                             1
            Iron, Fe                               1
            Magnesium, Mg                          1
            Manganese, Mn                          1
...                                              ...
{TRAIL MIX} Sodium, Na                             1
            Sugars, total including NLEA           1
            Total lipid (fat)                      1
            Vitamin A, IU                          1
            Vitamin C, total ascorbic acid         1

[4733021 rows x 1 columns]

## 2. HEALTH CARE.
### 2.1. Exploration.

-enfermedades carenciales
-cronicas
1. sacar informacion de las paginas web.
2. web scraping.

## que hipothesis? 
1. alguna mezcla de enfermedades, sacar pocas enfermedades y relacionarlo con la comida para ver que tipo de alimento necesits para recuperarte de cada enfermedad.
2. que nutriente tienen en comun la mayoria de los alimentos para ver cual es el plato o la comida mas saludable del mundo 
## cual es la comida mas sana del universo? 
    - which food is the helthier in the world
     - que nutriente es el mas popular 
            sacar un comun demnominador en la mayoria de comidas
    -  cual es la comida mas sana del mundo
            *sacar un porcentaje de cuales son los nutrientes mas populares 
            *buscar una comida que tenga la mayoria de los nutrientes 
            *agrupar los nutrientes? 
            *sacar porcentaje de nutrientes en food
            *buscar en las enfermedades cuales son los nutrientes mas usados.
                buscar que comida tiene la mayoria de esos nutrientes.
                sacar porcentajes count()sum() pero relacionados.



In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import lxml
import importlib
import src.extraction as ex
importlib.reload(ex)

<module 'src.extraction' from 'd:\\ironhack\\proyectos\\PipeLine_proy2\\src\\extraction.py'>

The total of the url link its not complete, its divided and reloaded every time I scroll down, will have to look for all the url links by separate and add them in a dataftame by separate, will apply a function that adds every link and make the same acctions in each.
After merge them all.

#### 2.1.1 First link.

get the text of what we want in the web page link.

In [5]:
url = "https://www.vedantu.com/biology/deficiency-diseases"
tags = ex.get_text(url)

In [6]:
tag_1 = tags[37:55]
tag_2 = tags[60:68]
tag_3 = tags[73:91]

clean and filter the text to what actualy we need.

In [7]:
info_1 = ex.text_filtered(tag_1)
info_2 = ex.text_filtered(tag_2)
info_3 = ex.text_filtered(tag_3)

Creating the lists info a data frame, it made it into a single column frame, had to separate the information into 2 columns, nutrients and the information, and at the end put the 3 data frames into 1.

In [8]:
df = ex.db_concat (info_1, info_2, info_3)
df

,nutrients,info
0,thiamine (vitamin b1),severe deficiency causes beriberi with symptom...
1,riboflavin (vitamin b2),the deficiency of this vitamin causes angular ...
2,niacin (vitamin b3),causes pellagra which is a life-threatening di...
3,pantothenic acid (vitamin b5),vitamin b5 deficiency is rare and the symptoms...
4,vitamin b6,end-stage kidney diseases such as celiac disea...
5,biotin (vitamin b7),the deficiency affects the growth of hair and ...
6,folate (vitamin b9),it is very common and the deficiency is associ...
7,vitamin b12,deficiency of this disease leads to pernicious...
8,ascorbic acid (vitamin c),deficiency results in scurvy and the symptoms ...
0,retinol (vitamin a),the deficiency causes night blindness or nycta...


df1

In [51]:
lsttotal

['severe deficiency causes beriberi with symptoms being weight loss, impaired sensory perception, weakness, and pain in limbs. some forms of beriberi are wernicke encephalopathy and korsakoff syndrome.',
 'the deficiency of this vitamin causes angular cheilitis which causes inflammation at mouth corners, chapped, and cracked lips. it can also cause anaemia.',
 'causes pellagra which is a life-threatening disease. the symptoms are diarrhoea, dermatitis, and dementia. if not treated at an early stage, the patient may die',
 'vitamin b5 deficiency is rare and the symptoms are fatigue, apathy, and irritability.',
 'end-stage kidney diseases such as celiac disease, crohn disease, etc. the symptoms include microcytic anaemia depression, dermatitis, and electroencephalographic abnormalities',
 'the deficiency affects the growth of hair and deteriorates skin health. increased urinary excretion of 3-hydroxyisovaleric acid is a sign of biotin deficiency.',
 'it is very common and the deficiency 

In [95]:
lsttotal = list(df["info"])
lst2 = lsttotal[0]
lst3 = lsttotal[1]
lst2 = lsttotal[2]
lst3 = lsttotal[3]
lst2 = lsttotal[4]
lst3 = lsttotal[15]
lst2 = lsttotal[21]
lst3 = lsttotal[15]
lst2 = lsttotal[21]
lst3 = lsttotal[15]
lst2 = lsttotal[21]
lst3 = lsttotal[15]

In [113]:
y = []
for i in lsttotal:
    new_list = []
    for e in disease:
        if e in i:
            new_list.append(i)
        else:
            pass
        y.append(new_list)
print(new_list)

['manganese is important for bone growth and cartilage development. a deficiency can lead to teeth grinding, convulsions, muscle contractions, vision and hearing problems in children. it causes hypertension, atherosclerosis and heart diseases in adults', 'manganese is important for bone growth and cartilage development. a deficiency can lead to teeth grinding, convulsions, muscle contractions, vision and hearing problems in children. it causes hypertension, atherosclerosis and heart diseases in adults', 'manganese is important for bone growth and cartilage development. a deficiency can lead to teeth grinding, convulsions, muscle contractions, vision and hearing problems in children. it causes hypertension, atherosclerosis and heart diseases in adults']


In [116]:
disease = ["beriberi","weight loss", "weakness", "angular cheilitis", "chapped", "anemia", "cracked lips", "pellagra", "diarrhoea", "dermatitis", "dementia", "fatigue", "apathy", "irritability", "celiac disease", "crohn", "skin health", "neural tube defects", "methylmalonic acidemia", "scurvy", "skin spots", "nyctalopia", "keratomalacia", "blindness", "rickets", "weakening of bones", "decreased coagulation of blood", "muscle weakness", "nerve weakness", "osteopenia", "nausea", "vomiting", "kidney disease", "loss of taste, smell", "goitre", "hyponatremia", "dehydration", "heart diseases", "muscle contractions", "hypertension"]
y = []
z = []
for i in disease:
    new_list = []
    for e in lsttotal:
        if i in e:
           new_list.append(i)
        else:
            pass
        y.append(new_list)


In [106]:
disease = ["beriberi","weight loss", "weakness", "angular cheilitis", "chapped", "anemia", "cracked lips", "pellagra", "diarrhoea", "dermatitis", "dementia", "fatigue", "apathy", "irritability", "celiac disease", "crohn", "skin health", "neural tube defects", "methylmalonic acidemia", "scurvy", "skin spots", "nyctalopia", "keratomalacia", "blindness", "rickets", "weakening of bones", "decreased coagulation of blood", "muscle weakness", "nerve weakness", "osteopenia", "nausea", "vomiting", "kidney disease", "loss of taste, smell", "goitre", "hyponatremia", "dehydration", "heart diseases", "muscle contractions", "hypertension"]
y = []
z = []
for i in disease:
    new_list = []
    if i in lst2:
        new_list.append(i)
    y.append(new_list)
print(y)

[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]


In [ ]:
lst2 = lsttotal[21]
lst3 = lsttotal[15]

In [117]:
disease = ["beriberi","weight loss", "weakness", "angular cheilitis", "chapped", "anemia", "cracked lips", "pellagra", "diarrhoea", "dermatitis", "dementia", "fatigue", "apathy", "irritability", "celiac disease", "crohn", "skin health", "neural tube defects", "methylmalonic acidemia", "scurvy", "skin spots", "nyctalopia", "keratomalacia", "blindness", "rickets", "weakening of bones", "decreased coagulation of blood", "muscle weakness", "nerve weakness", "osteopenia", "nausea", "vomiting", "kidney disease", "loss of taste, smell", "goitre", "hyponatremia", "dehydration", "heart diseases", "muscle contractions", "hypertension"]
z = []
for i in disease:
    x = []
    for e in range(0,22):
        lst2 = lsttotal[e]
        x.append(lst2)
        if i in x:
            z.append(i)
        else:
            pass
print(z)

[]


In [77]:
w = [z] + [y]
w

[[['weakness'], ['nausea'], ['vomiting']],
 [['heart diseases'], ['muscle contractions'], ['hypertension']]]

In [88]:
for i in df["info"]:
    for e in disease:
        if e in i:
            print(e)

beriberi
weight loss
weakness
angular cheilitis
chapped
cracked lips
pellagra
diarrhoea
dermatitis
dementia
fatigue
apathy
irritability
dermatitis
celiac disease
crohn
kidney disease
skin health
neural tube defects
methylmalonic acidemia
scurvy
skin spots
nyctalopia
keratomalacia
blindness
rickets
weakening of bones
decreased coagulation of blood
weakness
muscle weakness
nerve weakness
fatigue
osteopenia
weakness
nausea
vomiting
vomiting
kidney disease
loss of taste, smell
goitre
hyponatremia
dehydration
heart diseases
heart diseases
muscle contractions
hypertension


In [73]:
disease = ["beriberi","weight loss", "weakness", "angular cheilitis", "chapped", "anemia", "cracked lips", "pellagra", "diarrhoea", "dermatitis", "dementia", "fatigue", "apathy", "irritability", "celiac disease", "crohn", "skin health", "neural tube defects", "methylmalonic acidemia", "scurvy", "skin spots", "nyctalopia", "keratomalacia", "blindness", "rickets", "weakening of bones", "decreased coagulation of blood", "muscle weakness", "nerve weakness", "osteopenia", "nausea", "vomiting", "kidney disease", "loss of taste, smell", "goitre", "hyponatremia", "dehydration", "heart diseases", "muscle contractions", "hypertension"]
y = []
for i in disease:
    for e in lsttotal:
        if i in e:
            y.append([i])
        else:
            pass
print(y)

[['beriberi'], ['weight loss'], ['weakness'], ['weakness'], ['weakness'], ['angular cheilitis'], ['chapped'], ['cracked lips'], ['pellagra'], ['diarrhoea'], ['dermatitis'], ['dermatitis'], ['dementia'], ['fatigue'], ['fatigue'], ['apathy'], ['irritability'], ['celiac disease'], ['crohn'], ['skin health'], ['neural tube defects'], ['methylmalonic acidemia'], ['scurvy'], ['skin spots'], ['nyctalopia'], ['keratomalacia'], ['blindness'], ['rickets'], ['weakening of bones'], ['decreased coagulation of blood'], ['muscle weakness'], ['nerve weakness'], ['osteopenia'], ['nausea'], ['vomiting'], ['vomiting'], ['kidney disease'], ['kidney disease'], ['loss of taste, smell'], ['goitre'], ['hyponatremia'], ['dehydration'], ['heart diseases'], ['heart diseases'], ['muscle contractions'], ['hypertension']]


In [74]:
disease = ["beriberi","weight loss", "weakness", "angular cheilitis", "chapped", "anemia", "cracked lips", "pellagra", "diarrhoea", "dermatitis", "dementia", "fatigue", "apathy", "irritability", "celiac disease", "crohn", "skin health", "neural tube defects", "methylmalonic acidemia", "scurvy", "skin spots", "nyctalopia", "keratomalacia", "blindness", "rickets", "weakening of bones", "decreased coagulation of blood", "muscle weakness", "nerve weakness", "osteopenia", "nausea", "vomiting", "kidney disease", "loss of taste, smell", "goitre", "hyponatremia", "dehydration", "heart diseases", "muscle contractions", "hypertension"]
y = []
for i in disease:
    for row in df.info:
        if i in row:
            df.diseases = 
        else:
            pass
print(y)

SyntaxError: invalid syntax (3206439862.py, line 6)

In [21]:
for e in df.info:
    print(e)

TypeError: 'method' object is not iterable

### importar csv de alimentacion